In [2]:
import torch
import numpy as np
import os
import random
import pandas as pd
from astropy.io import fits
from case_studies.galaxy_clustering.utils import diagnostics
import matplotlib.pyplot as plt
from astropy.visualization import make_lupton_rgb
from astropy.io import fits
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from astropy.table import Table

In [13]:
image_g = fits.getdata("/data/scratch/des/new_psf_data/galsim_des_000_g.fits")
image_r = fits.getdata("/data/scratch/des/new_psf_data/galsim_des_000_r.fits")
image_i = fits.getdata("/data/scratch/des/new_psf_data/galsim_des_000_i.fits")

In [ ]:
image_g.shape

In [ ]:
rgb_default = make_lupton_rgb(image_i, image_r, image_g)
plt.figure(figsize=(10,10))
plt.imshow(rgb_default)

In [16]:
image_g = fits.getdata("/data/scratch/des/piff_test_data/images/galsim_des_000_g.fits")
image_r = fits.getdata("/data/scratch/des/piff_test_data/images/galsim_des_000_r.fits")
image_i = fits.getdata("/data/scratch/des/piff_test_data/images/galsim_des_000_i.fits")

In [ ]:
rgb_default = make_lupton_rgb(image_i, image_r, image_g)
plt.figure(figsize=(10,10))
plt.imshow(rgb_default)

In [2]:
import os

In [ ]:
DES_PATH = "/nfs/turbo/lsa-regier/scratch/gapatron/psf-models/dr2_tiles"
DES_TILES = os.listdir("/nfs/turbo/lsa-regier/scratch/gapatron/psf-models/dr2_tiles")
for DES_TILE_PATH in DES_TILES:
    files = os.listdir(f"{DES_PATH}/{DES_TILE_PATH}")
    print(files)
    break

In [12]:
NEW_DES_PATH =  "/nfs/turbo/lsa-regier/scratch/gapatron/psf-models/dr2_tiles_reformatted"
os.mkdir(NEW_DES_PATH)

In [13]:
NEW_DES_R_PATH =  "/nfs/turbo/lsa-regier/scratch/gapatron/psf-models/dr2_tiles_reformatted/rband"
NEW_DES_G_PATH =  "/nfs/turbo/lsa-regier/scratch/gapatron/psf-models/dr2_tiles_reformatted/gband"
NEW_DES_I_PATH =  "/nfs/turbo/lsa-regier/scratch/gapatron/psf-models/dr2_tiles_reformatted/iband"
NEW_DES_Z_PATH =  "/nfs/turbo/lsa-regier/scratch/gapatron/psf-models/dr2_tiles_reformatted/zband"

os.mkdir(NEW_DES_G_PATH)
os.mkdir(NEW_DES_R_PATH)
os.mkdir(NEW_DES_I_PATH)
os.mkdir(NEW_DES_Z_PATH)

In [ ]:
import shutil
for i, DES_TILE_PATH in enumerate(DES_TILES):
    files = os.listdir(f"{DES_PATH}/{DES_TILE_PATH}")
    rpsf = [file for file in files if file.endswith("r_nobkg.psf")][0]
    source_file = f"{DES_PATH}/{DES_TILE_PATH}/{rpsf}"
    dest_file = f"{NEW_DES_R_PATH}/galsim_des_{i:03d}_r.psf"
    shutil.copy(source_file, dest_file)
    print(f"Directory {i} done!")

In [ ]:
import shutil
for i, DES_TILE_PATH in enumerate(DES_TILES):
    files = os.listdir(f"{DES_PATH}/{DES_TILE_PATH}")
    gpsf = [file for file in files if file.endswith("g_nobkg.psf")][0]
    source_file = f"{DES_PATH}/{DES_TILE_PATH}/{gpsf}"
    dest_file = f"{NEW_DES_G_PATH}/galsim_des_{i:03d}_g.psf"
    shutil.copy(source_file, dest_file)
    print(f"Directory {i} done!")

In [ ]:
import shutil
for i, DES_TILE_PATH in enumerate(DES_TILES):
    files = os.listdir(f"{DES_PATH}/{DES_TILE_PATH}")
    ipsf = [file for file in files if file.endswith("i_nobkg.psf")][0]
    source_file = f"{DES_PATH}/{DES_TILE_PATH}/{ipsf}"
    dest_file = f"{NEW_DES_I_PATH}/galsim_des_{i:03d}_i.psf"
    shutil.copy(source_file, dest_file)
    print(f"Directory {i} done!")

In [ ]:
import shutil
for i, DES_TILE_PATH in enumerate(DES_TILES):
    files = os.listdir(f"{DES_PATH}/{DES_TILE_PATH}")
    zpsf = [file for file in files if file.endswith("z_nobkg.psf")][0]
    source_file = f"{DES_PATH}/{DES_TILE_PATH}/{zpsf}"
    dest_file = f"{NEW_DES_Z_PATH}/galsim_des_{i:03d}_z.psf"
    shutil.copy(source_file, dest_file)
    print(f"Directory {i} done!")

In [ ]:
source_directory = "/nfs/turbo/lsa-regier/scratch/gapatron/psf-models/dr2_tiles_reformatted/gband"
files = os.listdir(source_directory)
num_files = len(files)
for i, file in enumerate(files):
    og_path = f"{source_directory}/{file}"
    for j in range(5):
        new_index = (j+1) * num_files + i
        filename = f"galsim_des_{new_index:03d}_g.psf"
        new_file_path = f"{source_directory}/{filename}"
        shutil.copy(og_path, new_file_path)
    print(f"Made copies of file {i}")

In [ ]:
source_directory = "/nfs/turbo/lsa-regier/scratch/gapatron/psf-models/dr2_tiles_reformatted/rband"
files = os.listdir(source_directory)
num_files = len(files)
for i, file in enumerate(files):
    og_path = f"{source_directory}/{file}"
    for j in range(5):
        new_index = (j+1) * num_files + i
        filename = f"galsim_des_{new_index:03d}_r.psf"
        new_file_path = f"{source_directory}/{filename}"
        shutil.copy(og_path, new_file_path)
    print(f"Made copies of file {i}")

In [ ]:
source_directory = "/nfs/turbo/lsa-regier/scratch/gapatron/psf-models/dr2_tiles_reformatted/iband"
files = os.listdir(source_directory)
num_files = len(files)
for i, file in enumerate(files):
    og_path = f"{source_directory}/{file}"
    for j in range(5):
        new_index = (j+1) * num_files + i
        filename = f"galsim_des_{new_index:03d}_i.psf"
        new_file_path = f"{source_directory}/{filename}"
        shutil.copy(og_path, new_file_path)
    print(f"Made copies of file {i}")

In [ ]:
source_directory = "/nfs/turbo/lsa-regier/scratch/gapatron/psf-models/dr2_tiles_reformatted/zband"
files = os.listdir(source_directory)
num_files = len(files)
for i, file in enumerate(files):
    og_path = f"{source_directory}/{file}"
    for j in range(5):
        new_index = (j+1) * num_files + i
        filename = f"galsim_des_{new_index:03d}_z.psf"
        new_file_path = f"{source_directory}/{filename}"
        shutil.copy(og_path, new_file_path)
    print(f"Made copies of file {i}")